In [1]:
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, 'utils/')

from projection import Projection
from projection_old import Projection as Proj_old
from projection_old import *
from utils_challenge import *
from utils_projections import *
from np_ioueval import iouEval

Using TensorFlow backend.


In this notebook, we take as input the 2D prediction presented in the previous notebook (2) Prediction of spherical projection). It is loaded in cell 4. The output of this notebook is a fully labeled 3D point cloud.

In [2]:
selected_set = 'train'
file_name = '40'

In [3]:
# original downloaded data
path_to_gt = 'input_data/point_clouds/' + selected_set
path_to_dict = 'input_data/parameters/'


path_to_ground_labeled = 'output_data/ground_labeled/'
path_to_2d_preds = 'output_data/2d_preds/'
output_path_3d = 'output_data/3d_preds/'

In [4]:
pred = load_h5_file(path_to_2d_preds + file_name + '.h5') 

In [5]:
width_sph = 1024
height_sph = 64
fov_up = 50
fov_down = 115

proj_sph = Proj_old(proj_type='front', width=width_sph, height=height_sph, fov_up=fov_up, fov_down=fov_down)

In [6]:
def load_spherical_prediction(path_to_file, file_name):
    full_path = os.path.join(path_to_file, file_name + '.npy')
    return np.load(full_path)

def load_point_cloud(path_to_file, file_name, skip_names = False):
    full_path = os.path.join(path_to_file, file_name + '.txt')
    if skip_names:
        point_cloud = np.loadtxt(full_path, skiprows=1)
    else:
        point_cloud = np.loadtxt(full_path)
    return point_cloud

In [7]:
def compute_centered_point_cloud(points):
    min_val = points.min(0)

    xmin = min_val[0]
    ymin = min_val[1]
    zmin = min_val[2]

    centered_xyz = np.zeros_like(points)
    centered_xyz[:,0] = points[:, 0] - xmin
    centered_xyz[:,1] = points[:, 1] - ymin
    centered_xyz[:,2] = points[:, 2]    
    
    return centered_xyz

In [8]:
mapping_files = {
    # training set
"00": "5D4KVPBP",
"01": "5D4KVPC9",
"02": "5D4KVPDO",
"03": "5D4KVPFI",
"04": "5D4KVPJE",
"05": "5D4KVPNC",
"06": "5D4KVPT5",
"07": "5D4KVPX2",
"08": "5D4KVQ0P",
"09": "5D4KVRER",
"10": "5D4KVRWC",
"11": "5D4KVT48",
"12": "5D4KVT9X",
"13": "5D4KX2ZN",
"14": "5D4KX3LW",
"15": "5D4KX3PZ",
"16": "5D4KX3VN",
"17": "5D4KX4HE",
"18": "5D4KX4QC",
"19": "5D4KX4ZE",
"20": "5D4KX56H",
"21": "5D4KX5NM",
"22": "5D4KX5WV",
"23": "5D4KX66R",
"24": "5D4KX6L3",
"25": "5D4KX6T5",
"26": "5D4KX7FN",
"27": "5D4KX7IA",
"28": "5D4KX7KT",
"29": "5D4KX7RD",
"30": "5D4KX826",
"31": "5D4KX8IR",
"32": "5D4KX8UQ",
"33": "5D4KX8Y6",
"34": "5D4KX993",
"35": "5D4KX9SY",
"36": "5D4KX9ZE",
"37": "5D4KXA0G",
"38": "5D4KXAW7",
"39": "5D4KXB8D",
"40": "5D4KXBC8",
"41": "5D4KXBTC",
"42": "5D4L1GZR",
"43": "5D4L1IQ4",
"44": "5D4L1M3I",
"45": "5D4L1QP2",
"46": "5D4L1RDR",
"47": "5D4L1TH9",
"48": "5D4L1TYC",
"49": "5D4L1WHI",
"50": "5D4L1XPJ",
"51": "5D4L1Y38",
"52": "5D4L1YDX",
"53": "5D4L2BFI",
"54": "5D4L2C9B",
"55": "5D4L2DGW",
"56": "5D4L2DTM",
"57": "5D4L2FRJ",
"58": "5D4L2G9K",
"59": "5D4LHQUX",
    #test set
"60": "5D4KVPG4",
"61": "5D4KVPIN",
"62": "5D4KVPXD",
"63": "5D4KVPYD",
"64": "5D4KVQ9U",
"65": "5D4KX38L",
"66": "5D4KX3EC",
"67": "5D4KX3RR",
"68": "5D4KX3TQ",
"69": "5D4KX40Y",
"70": "5D4KX5G9",
"71": "5D4KX76F",
"72": "5D4KX9N2",
"73": "5D4KX9SD",
"74": "5D4L1JIE",
"75": "5D4L1MGO",
"76": "5D4L1P8E",
"77": "5D4L1RW5",
"78": "5D4L1TDI",
"79": "5D4L1TX7"
}

In [9]:
dict_name = os.path.join(path_to_dict, selected_set + '_dict')
laser_positions = load_obj(dict_name)

print("loading point cloud:", file_name)
point_cloud_xyz = load_point_cloud(path_to_gt,  mapping_files[file_name], skip_names = True)[:,:3]
    
print("loading ground detected with lambda flat zones")
ground_lfz = load_h5_file(path_to_ground_labeled + file_name + '.h5')

loading point cloud: 40
loading ground detected with lambda flat zones


In [10]:
# idx of ground and no-ground from lambda flat zones method
no_ground_index = np.where(ground_lfz == 0)
ground_index = np.where(ground_lfz != 0)


centered_xyz = compute_centered_point_cloud(point_cloud_xyz)
xyz_to_sensor = centered_xyz - laser_positions[file_name]

xyz_to_sensor_no_ground = xyz_to_sensor[no_ground_index]

aux_idx_proj, mapping = min_aggregation_proj(64,1024,proj_sph,xyz_to_sensor_no_ground)

valid_3d_idx_in_2d = np.where(aux_idx_proj > -1)
idx_in_3d = aux_idx_proj[valid_3d_idx_in_2d]

# fill 3D labels
print("Labeling points using spherical prediction")
labels_3d_no_ground = np.zeros_like(no_ground_index[0])
labels_3d_no_ground[idx_in_3d] = pred[valid_3d_idx_in_2d]
    

# label "no-ground" points using DL model
labels_3d_pred = np.zeros(point_cloud_xyz.shape[0])
labels_3d_pred[no_ground_index] = labels_3d_no_ground

Labeling points using spherical prediction


In [11]:
print("Propagating labels to hidden ones, using knn with k = 3")

unlabeled_points_id = np.where(labels_3d_pred == 0)[0]
labeled_points_id = np.where(labels_3d_pred > 0)[0]

X_visible, y_visible = point_cloud_xyz[labeled_points_id, :], labels_3d_pred[labeled_points_id]
X_hidden = point_cloud_xyz[unlabeled_points_id, :]

num_neighbors = 3

knn = KNeighborsClassifier(n_neighbors=num_neighbors)
knn.fit(X_visible, y_visible)
y_pred_knn = knn.predict(X_hidden)

Propagating labels to hidden ones, using knn with k = 3


In [12]:
# assign propagated labels to non-labeled points
labels_3d_pred[unlabeled_points_id] = y_pred_knn

# label ground using lambda flat zones
labels_3d_pred[ground_index] = 3 # label of ground in this dataset    

In [13]:
out_pc = np.concatenate([point_cloud_xyz, labels_3d_pred.reshape(-1,1)], axis = 1)
ppp = output_path_3d + file_name + '.txt'
np.savetxt(ppp, out_pc)
print(ppp)
print("out_pc", out_pc.shape)

output_data/3d_preds/40.txt
out_pc (4460335, 4)


<b>Some observations: </b>

In this notebook we performed backprojection using 2d prediction from Deep Learning model. Slightly better results are obtained using KNN from penultimate layer from DL model.

Postprocessing step improves buildings and poles.

In [14]:
gt_labels = load_point_cloud(path_to_gt,  mapping_files[file_name], skip_names = True)[:,3]

In [15]:
from sklearn.metrics import classification_report, confusion_matrix

In [16]:
print(classification_report(gt_labels[gt_labels > 0], labels_3d_pred[gt_labels > 0]))

              precision    recall  f1-score   support

         1.0       0.93      0.79      0.86   1830101
         2.0       0.82      0.32      0.46    356583
         3.0       0.86      1.00      0.92   1556902
         4.0       0.00      0.00      0.00      2751
         5.0       0.65      0.87      0.74    710729

   micro avg       0.84      0.84      0.84   4457066
   macro avg       0.65      0.60      0.60   4457066
weighted avg       0.85      0.84      0.83   4457066



In [17]:
cm = confusion_matrix(gt_labels[gt_labels > 0], labels_3d_pred[gt_labels > 0])
print(cm)

[[1453254   17549   74521     705  284072]
 [  50427  115008  139210    1034   50904]
 [   2277    1621 1552551      10     443]
 [   2665       0       0       0      86]
 [  46621    5455   41825    1390  615438]]
